In [1]:
%matplotlib notebook
import os
os.chdir('C:/Users/ebray/OneDrive - NASA/Python Programs/SORC Poses/') #I don't understand why I JUST now have to add this to get this cell to work
# %matplotlib inline
from adm_library import *
import pandas as pd

## 0 Parameter definitions

### 0.1 Ingesting the [X,Y,Z,Rx,Ry] values from Tyler's spreadsheet

In [2]:
fname = 'SES-sandbox.xlsx'
sheetname = 'Input for Tyler analysis'
#fname = 'SES-cold plateau 3 - circuit 2.xlsx'
#sheetname = 'Input for Tyler analysis'

spreadsheet = pd.read_excel(fname,
                            sheet_name=sheetname,skiprows=1,usecols='B:N')

focal_length = spreadsheet.iloc[0][0]-110 # focal length is track length -110mm, per TG
sMPA_to_WCS = spreadsheet.iloc[3,:3].astype(float)
sMPA_angle_to_WCS_deg = spreadsheet.iloc[6,:3].astype(float)
GSA_angle_WCS_deg = spreadsheet.iloc[7][0]
sMask=np.array(spreadsheet.iloc[10,1:4]).astype(float)

spreadsheet = pd.read_excel(fname,
                            sheet_name=sheetname,skiprows=15,usecols='B:N')
pose_actual = spreadsheet.iloc[0:5,0:6]
pose_actual = pose_actual.set_index('Name').sort_index()
pose_actual.columns = ['X','Y','Z','Rx','Ry']
X_5DOF,Y_5DOF,Z_5DOF,rx_deg_5DOF,ry_deg_5DOF = [[*pose_actual['X'].values.astype(float)],[*pose_actual['Y'].values.astype(float)],[*pose_actual['Z'].values.astype(float)],
                                                [*pose_actual['Rx'].values.astype(float)],[*pose_actual['Ry'].values.astype(float)]]
pose_encoders = spreadsheet.iloc[0:5,7:13]
pose_encoders = pose_encoders.set_index('Name.1').sort_index()

print('Provided 5DOF position in real space: \n',pose_actual,'\n')
print('Provided 5DOF encoder values: \n',pose_encoders)


Provided 5DOF position in real space: 
             X       Y       Z      Rx       Ry
Name                                          
PDI1 -109.154  -4.241  21.282 -0.9354  10.8024
PDI2  111.366  -3.589  21.566 -0.8781 -10.7867
PDI3   39.731  52.251  10.951  4.7478  -3.8001
PDI4  -38.875  52.142  11.213  4.7342   3.9592
PR1      0.88   1.521  -0.439  0.0376   0.0332 

Provided 5DOF encoder values: 
               X       Y       Z      Rx       Ry
Name.1                                          
PDI1   -109.126  -4.198  21.349 -0.9269  10.8034
PDI2    111.246  -3.505  21.717 -0.8830 -10.7845
PDI3     39.667  52.276  11.079  4.7475  -3.8045
PDI4    -38.887  52.153  11.312  4.7387   3.9557
PR1       0.854   1.555  -0.386  0.0395   0.0340


### 0.1.1 If both the pose_actual and pose_encoder arrays are provided in the spreadsheet above, convert between the two to make sure they both agree. 

In [3]:
#No point doing the following if one or the other array is completely empty
if (not any(np.isfinite(pose_encoders.values.astype(float)).ravel())) and (not any(np.isfinite(pose_actual.values.astype(float)).ravel())):

    print('(Calculated 5DOF position) - (Actual 5DOF position (from the spreadsheet)): ')
    calculated_pose_actual = pd.DataFrame(columns=pose_actual.columns)
    for index in [val for val in pose_actual.index if ('PR' in val) or ('PD' in val)]:
        calculated_pose_actual = pd.concat((calculated_pose_actual,calculate_5DOF_from_encoders(pd.DataFrame(pose_encoders.loc[index]).T)))
    print(calculated_pose_actual-pose_actual,'\n\n')
    
    print('(Calculated 5DOF encoders) - (Actual 5DOF encoders (from the spreadsheet)): ')
    calculated_pose_encoders = pd.DataFrame(columns=pose_encoders.columns)
    for index in [val for val in pose_encoders.index if ('PR' in val) or ('PD' in val)]:
        calculated_pose_encoders = pd.concat((calculated_pose_encoders,calculate_encoders_from_5DOF(pd.DataFrame(pose_actual.loc[index]).T)))
    print(calculated_pose_encoders-pose_encoders)

### 0.2 If the "pose_actual" section of the supplied spreadsheet was empty, build a new "pose_actual" array based on the input "pose_encoder" array.

In [4]:
if not any(np.isfinite(pose_actual.values.astype(float)).ravel()):
    pose_actual = convert_pose_encoders_to_pose_actual(pose_encoders)
    print('NEWLY calculated 5DOF position in real space based on recorded 5DOF encoder values: \n',pose_actual)

# 1. Updating Poses with ADM Data

## 1.1 ADM Baseline  

#### 1.1.1 Input your baseline ADM values

In [5]:
# 4/1 SES cold
z_null_sMATF_baseline = 4.027174*1000. 
z_null_PATB_baseline = 3.455684*1000.
sMATF_baseline = pd.DataFrame(dict({'AC_AZ':0.0041,'AC_EL':0.006,'DET_X':811.9,'DET_Y':604.6}),index=['sMATF'])
p_null_PATB_baseline_encoder = pd.DataFrame(dict({'X':88.435, 'Y':109.266,'Z':0,'Rx':0.1018,'Ry':0.0856,
                                                  'AC_AZ':0.0041,'AC_EL':0.0035,'DET_X':812.6,'DET_Y':605.9}),index=['PATB'])

p_null_PATB_baseline = calculate_5DOF_from_encoders(p_null_PATB_baseline_encoder)

p_null_offset = np.array([-90.,-110.,-10.]) #PATB is not on the POA boresight. It is offset by this much
p_null_PATB_baseline_encoder


,X,Y,Z,Rx,Ry,AC_AZ,AC_EL,DET_X,DET_Y
PATB,88.435,109.266,0,0.1018,0.0856,0.0041,0.0035,812.6,605.9


### 1.1.2 Update the PATB pose to account for imperfect nulling.

In [6]:

optimal_gsarx_patb, optimal_gsary_patb = optimize_5DOF_rotation_for_PAT(p_null_PATB_baseline_encoder['AC_AZ'].values[0], p_null_PATB_baseline_encoder['AC_EL'].values[0], 
                                         sMATF_baseline['AC_AZ'].values[0],sMATF_baseline['AC_EL'].values[0], 
                                         p_null_PATB_baseline_encoder['Rx'].values[0], p_null_PATB_baseline_encoder['Ry'].values[0])
print('Baseline GSA RX/RY values: ', p_null_PATB_baseline_encoder['Rx'].values[0], p_null_PATB_baseline_encoder['Ry'].values[0])
print('Optimal GSA RX/Ry values: ',optimal_gsarx_patb,optimal_gsary_patb)
additional_patb_z_shift = -1*p_null_offset[1]*np.sin(np.deg2rad(optimal_gsarx_patb - p_null_PATB_baseline_encoder['Rx'].values[0])) +\
                            p_null_offset[0]*np.sin(np.deg2rad(optimal_gsary_patb - p_null_PATB_baseline_encoder['Ry'].values[0]))
z_null_PATB_baseline += additional_patb_z_shift
print('Effective change in Z-position of PATB after applying optimal rotations: ',round(additional_patb_z_shift,4))


optimal_htsax_patb, optimal_vtsa_patb = optimize_5DOF_translation_for_PAT(p_null_PATB_baseline_encoder['DET_X'].values[0], p_null_PATB_baseline_encoder['DET_Y'].values[0], 
                                         sMATF_baseline['DET_X'].values[0],sMATF_baseline['DET_Y'].values[0], 
                                         p_null_PATB_baseline_encoder['X'].values[0], p_null_PATB_baseline_encoder['Y'].values[0])
print('Baseline HTSAX/VTSA values: ',p_null_PATB_baseline_encoder['X'].values[0], p_null_PATB_baseline_encoder['Y'].values[0])
print('Optimal HTSAX/VTSA values: ',optimal_htsax_patb,optimal_vtsa_patb)

p_null_PATB_baseline_encoder_alternate = pd.DataFrame(dict({'X':optimal_htsax_patb, 'Y':optimal_vtsa_patb,'Z':p_null_PATB_baseline_encoder['Z'].values[0],'Rx':optimal_gsarx_patb,'Ry':optimal_gsary_patb,
                                                  'AC_AZ':0,'AC_EL':0,'DET_X':sMATF_baseline['DET_X'].values[0],'DET_Y':sMATF_baseline['DET_Y'].values[0]}),index=['PATB'])
p_null_PATB_baseline = calculate_5DOF_from_encoders(p_null_PATB_baseline_encoder_alternate)
print('Updated PATB baseline pose: \n',p_null_PATB_baseline_encoder_alternate.loc['PATB',['X','Y','Z','Rx','Ry']])


Baseline GSA RX/RY values:  0.1018 0.0856
Optimal GSA RX/Ry values:  0.1032160175 0.08766031499999999
Effective change in Z-position of PATB after applying optimal rotations:  -0.0005
Baseline HTSAX/VTSA values:  88.435 109.266
Optimal HTSAX/VTSA values:  88.446418 109.33648500000001
Updated PATB baseline pose: 
 X      88.446418
Y     109.336485
Z       0.000000
Rx      0.103216
Ry      0.087660
Name: PATB, dtype: float64


#### 1.1.5 Form a dataframe with the values supplied so far

In [7]:
#Generate a dataframe where we will store some parameters for each pose
columns = ['X','Y','Z','Rx','Ry']
df = pose_actual
df['color']='crimson'

gsa_rot = R.from_euler('x',GSA_angle_WCS_deg, degrees=True)
df.loc['sMask',['X','Y','Z']] = gsa_rot.apply(sMask)
df.loc['sMask','color'] = 'g'

dz_baseline = z_null_sMATF_baseline-z_null_PATB_baseline #deltaZ between PATB and sMATF

df = pd.concat([df,p_null_PATB_baseline])
df.loc['PATB','color']='darkblue'

df.loc['sMPA',['X','Y','Z']] = sMPA_to_WCS.values.astype(float)
df.loc['sMPA','color']='purple'

for pose in df.index:
    update_uvec(df,pose,length=focal_length,rotangle=GSA_angle_WCS_deg)
    if 'PDI' in pose:
        df.loc[pose,'color'] = 'yellow'

# create normal vector to sMPA
vec = np.array([0,0,1])
rotmat = R.from_euler('XYZ',sMPA_angle_to_WCS_deg, degrees=True)
df.loc['sMPA',['uvec_X','uvec_Y','uvec_Z']] = rotmat.apply(vec)

#ax = plot_poses(df)
#plot_sMPA(df, ax)
df

,X,Y,Z,Rx,Ry,color,uvec_X,uvec_Y,uvec_Z
PDI1,-109.154,-4.241,21.282,-0.935400,10.802400,yellow,242.172872,264.729711,1241.310730
PDI2,111.366,-3.589,21.566,-0.878100,-10.786700,yellow,-241.825074,263.501943,1241.639725
PDI3,39.731,52.251,10.951,4.747800,-3.800100,yellow,-85.636281,142.724218,1281.357902
PDI4,-38.875,52.142,11.213,4.734200,3.959200,yellow,89.216049,143.001431,1281.082720
PR1,0.88,1.521,-0.439,0.037600,0.033200,crimson,0.748720,248.007345,1268.098436
sMask,1.228,116.421532,587.065659,NaN,NaN,g,NaN,NaN,NaN
PATB,88.536864,109.347797,-0.14863,0.101005,0.086966,darkblue,1.961247,246.603641,1268.370863
sMPA,1.6298,249.5191,1267.639,NaN,NaN,purple,0.000411,0.411286,0.911506


#### 1.1.6 Calculate the center of the sMATF flat in 5DOF coordinate system (currently assumes PATB is on-axis and aligned with GSA)

In [8]:
start_coords = np.array(df.loc['PATB',['X','Y','Z']])

rx = df.loc['PATB','Rx']
ry = df.loc['PATB','Ry']
gsa_rot = R.from_euler('XY',[rx+GSA_angle_WCS_deg,ry], degrees=True)
uvec = np.array([0.,0.,dz_baseline]+p_null_offset)
rot_uvec = gsa_rot.apply(uvec) #np.dot(gsa_rot.as_matrix(),uvec)

sMATF_coords = start_coords+rot_uvec

df.loc['sMATF',['X','Y','Z']]=sMATF_coords
df.loc['sMATF',['Rx','Ry']]=df.loc['PATB',['Rx','Ry']]
df.loc['sMATF','color']='orange'
update_uvec(df,'sMATF',length=dz_baseline,rotangle=GSA_angle_WCS_deg)
#ax = plot_poses(df)
#plot_sMPA(df, ax)
df

,X,Y,Z,Rx,Ry,color,uvec_X,uvec_Y,uvec_Z
PDI1,-109.154,-4.241,21.282,-0.935400,10.802400,yellow,242.172872,264.729711,1241.310730
PDI2,111.366,-3.589,21.566,-0.878100,-10.786700,yellow,-241.825074,263.501943,1241.639725
PDI3,39.731,52.251,10.951,4.747800,-3.800100,yellow,-85.636281,142.724218,1281.357902
PDI4,-38.875,52.142,11.213,4.734200,3.959200,yellow,89.216049,143.001431,1281.082720
PR1,0.88,1.521,-0.439,0.037600,0.033200,crimson,0.748720,248.007345,1268.098436
sMask,1.228,116.421532,587.065659,NaN,NaN,g,NaN,NaN,NaN
PATB,88.536864,109.347797,-0.14863,0.101005,0.086966,darkblue,1.961247,246.603641,1268.370863
sMPA,1.6298,249.5191,1267.639,NaN,NaN,purple,0.000411,0.411286,0.911506
sMATF,-0.610775,108.557113,572.148212,0.101005,0.086966,orange,0.867436,109.069835,560.985232


## 1.2 ADM Update

Now the system has undergone some kind of change that will cause the ADM measurements to be different, such as shimming the sWFI or cooling from ambient to operational temps. We need to use the new set of ADM sMATF/PATB measurements to update the calculated poses for PRs 1-5.

Now update all poses from this new PATB pose; this is the recipe that would be followed for real data

#### 1.2.1 Input new ADM data for the sMATF and PATB and convert to 5DOF actual space

In [9]:
# 4/3, use mirror for range if possible
z_null_sMATF_update = 4.031636*1000. 
z_null_PATB_update = 3.458517*1000.

sMATF_update = pd.DataFrame(dict({'AC_AZ':-0.0022,'AC_EL':0.0042,'DET_X':811.7,'DET_Y':597.6}),index=['sMATF'])
p_null_PATB_update_encoder = pd.DataFrame(dict({'X':88.526, 'Y':108.364,'Z':0.00,'Rx':0.1287,'Ry':0.0788,
                                                  'AC_AZ':-0.0008,'AC_EL':0.0001,'DET_X':812.1,'DET_Y':598.8}),index=['PATB_update'])

p_null_PATB_update = calculate_5DOF_from_encoders(p_null_PATB_update_encoder)
p_null_PATB_update

,X,Y,Z,Rx,Ry
PATB_update,88.616474,108.374721,-0.148095,0.126482,0.078135


### 1.2.2 Update the PATB pose to account for imperfect nulling

In [10]:
optimal_gsarx_patb, optimal_gsary_patb = optimize_5DOF_rotation_for_PAT(p_null_PATB_update_encoder['AC_AZ'].values[0], p_null_PATB_update_encoder['AC_EL'].values[0], 
                                         sMATF_update['AC_AZ'].values[0],sMATF_update['AC_EL'].values[0], 
                                         p_null_PATB_update_encoder['Rx'].values[0], p_null_PATB_update_encoder['Ry'].values[0])
print('Update GSA RX/RY values: ', p_null_PATB_update_encoder['Rx'].values[0], p_null_PATB_update_encoder['Ry'].values[0])
print('Optimal GSA RX/Ry values: ',optimal_gsarx_patb,optimal_gsary_patb)
additional_patb_z_shift = -1*p_null_offset[1]*np.sin(np.deg2rad(optimal_gsarx_patb - p_null_PATB_update_encoder['Rx'].values[0])) +\
                            p_null_offset[0]*np.sin(np.deg2rad(optimal_gsary_patb - p_null_PATB_update_encoder['Ry'].values[0]))
z_null_PATB_update += additional_patb_z_shift
print('Effective change in Z-position of PATB after applying optimal rotations: ',round(additional_patb_z_shift,4))


optimal_htsax_patb, optimal_vtsa_patb = optimize_5DOF_translation_for_PAT(p_null_PATB_update_encoder['DET_X'].values[0], p_null_PATB_update_encoder['DET_Y'].values[0], 
                                         sMATF_update['DET_X'].values[0],sMATF_update['DET_Y'].values[0], 
                                         p_null_PATB_update_encoder['X'].values[0], p_null_PATB_update_encoder['Y'].values[0])
print('update HTSAX/VTSA values: ',p_null_PATB_update_encoder['X'].values[0], p_null_PATB_update_encoder['Y'].values[0])
print('Optimal HTSAX/VTSA values: ',optimal_htsax_patb,optimal_vtsa_patb)

p_null_PATB_update_encoder_alternate = pd.DataFrame(dict({'X':optimal_htsax_patb, 'Y':optimal_vtsa_patb,'Z':p_null_PATB_update_encoder['Z'].values[0],'Rx':optimal_gsarx_patb,'Ry':optimal_gsary_patb,
                                                  'AC_AZ':0,'AC_EL':0,'DET_X':sMATF_update['DET_X'].values[0],'DET_Y':sMATF_update['DET_Y'].values[0]}),index=['PATB_update'])
p_null_PATB_update = calculate_5DOF_from_encoders(p_null_PATB_update_encoder_alternate)
print('Updated PATB update pose: \n',p_null_PATB_update_encoder_alternate.loc['PATB_update',['X','Y','Z','Rx','Ry']])


Update GSA RX/RY values:  0.1287 0.0788
Optimal GSA RX/Ry values:  0.1298684923 0.0829718906
Effective change in Z-position of PATB after applying optimal rotations:  -0.0043
update HTSAX/VTSA values:  88.526 108.364
Optimal HTSAX/VTSA values:  88.54602399999999 108.421932
Updated PATB update pose: 
 X      88.546024
Y     108.421932
Z       0.000000
Rx      0.129868
Ry      0.082972
Name: PATB_update, dtype: float64


#### 1.2.2 Use the newly-provided data to calculate an updated position for sMATF

In [11]:
dz_update = z_null_sMATF_update-z_null_PATB_update
df = pd.concat([df,p_null_PATB_update])
df.loc['PATB_update','color'] = 'purple'
update_uvec(df,'PATB_update',length=focal_length,rotangle=GSA_angle_WCS_deg)

start_coords = np.array(df.loc['PATB_update',['X','Y','Z']])

rx = df.loc['PATB_update','Rx']
ry = df.loc['PATB_update','Ry']
gsa_rot = R.from_euler('XY',[rx+GSA_angle_WCS_deg,ry], degrees=True)
uvec = [0.,0.,dz_update]+p_null_offset # in GSA CS
rot_uvec = gsa_rot.apply(uvec) #np.dot(gsa_rot.as_matrix(),uvec)

sMATF_update_coords = start_coords+rot_uvec

df.loc['sMATF_update',['X','Y','Z']]=sMATF_update_coords
df.loc['sMATF_update',['Rx','Ry']]=df.loc['PATB_update',['Rx','Ry']]
df.loc['sMATF_update','color']='brown'
update_uvec(df,'sMATF_update',length=dz_update,rotangle=GSA_angle_WCS_deg)
#ax = plot_poses(df)
#plot_sMPA(df, ax)
print('Baseline poses')
df


Baseline poses


,X,Y,Z,Rx,Ry,color,uvec_X,uvec_Y,uvec_Z
PDI1,-109.154,-4.241,21.282,-0.935400,10.802400,yellow,242.172872,264.729711,1241.310730
PDI2,111.366,-3.589,21.566,-0.878100,-10.786700,yellow,-241.825074,263.501943,1241.639725
PDI3,39.731,52.251,10.951,4.747800,-3.800100,yellow,-85.636281,142.724218,1281.357902
PDI4,-38.875,52.142,11.213,4.734200,3.959200,yellow,89.216049,143.001431,1281.082720
PR1,0.88,1.521,-0.439,0.037600,0.033200,crimson,0.748720,248.007345,1268.098436
sMask,1.228,116.421532,587.065659,NaN,NaN,g,NaN,NaN,NaN
PATB,88.536864,109.347797,-0.14863,0.101005,0.086966,darkblue,1.961247,246.603641,1268.370863
sMPA,1.6298,249.5191,1267.639,NaN,NaN,purple,0.000411,0.411286,0.911506
sMATF,-0.610775,108.557113,572.148212,0.101005,0.086966,orange,0.867436,109.069835,560.985232
PATB_update,88.636513,108.432684,-0.148136,0.127647,0.082307,purple,1.856177,246.013860,1268.485547


In [12]:
#How did the distance between PATB and sMATF change from pre- to post-environment shift?
print('PATB->sMATF distance changed by: ',dz_update-dz_baseline, ' mm')

PATB->sMATF distance changed by:  1.63279206946072  mm


## 1.3 Calculate deltas between baseline & updated ADM measurements

In [13]:
sMATF_delta = df.loc['sMATF_update',['X','Y','Z']] - df.loc['sMATF',['X','Y','Z']]
print('sMATF (X,Y,Z) deltas: \n',sMATF_delta,'\n')

patb_delta = df.loc['PATB_update',['X','Y','Z','Rx','Ry']] - df.loc['PATB',['X','Y','Z','Rx','Ry']]
#patb_drxry = df.loc['PATB_update',['Rx','Ry']] - df.loc['PATB',['Rx','Ry']]
print('PATB (X,Y,Z,Rx,Ry) deltas: \n',patb_delta,'\n')


sMATF (X,Y,Z) deltas: 
 X    0.056325
Y   -0.871732
Z    1.595869
dtype: object 

PATB (X,Y,Z,Rx,Ry) deltas: 
 X     0.099649
Y    -0.915114
Z     0.000494
Rx    0.026642
Ry   -0.004659
dtype: object 



## 1.4 Transform old poses to the new sMATF reference frame

### 1.4.1 Apply a translation to all poses

In [14]:
df_update = df.copy()
T = df.loc['sMATF_update',['X','Y','Z']]
df_update[['X','Y','Z']] -= T
print('Translation required to put sMATF_update at the origin: \n',T)

Translation required to put sMATF_update at the origin: 
 X    -0.554449
Y    107.68538
Z    573.74408
Name: sMATF_update, dtype: object


### 1.4.2 Now perform 3 rotations to all the poses:
1. To align to the sMATF frame
2. To rotate everything by delta(Rx,Ry)
3. To undo #1

In [15]:
update_rot1 = R.from_euler('XY',[df.loc['PATB','Rx']+GSA_angle_WCS_deg,df.loc['PATB','Ry']], degrees=True)
update_rot2 = R.from_euler('XY',[patb_delta.loc['Rx'],patb_delta.loc['Ry']], degrees=True)
update_rot_combined = update_rot1 * update_rot2 * update_rot1.inv()
                                
for pose in df_update.index:
                                
    df_update.loc[pose,['X','Y','Z']]=update_rot_combined.apply(df_update.loc[pose,['X','Y','Z']].astype(float)) #np.dot(update_rot.as_matrix(),df_update.loc[pose,['X','Y','Z']])
    df_update.loc[pose,['uvec_X','uvec_Y','uvec_Z']] = update_rot_combined.apply(df_update.loc[pose,['uvec_X','uvec_Y','uvec_Z']].astype(float)) 
    if pose not in ['sMask','sMPA']:
        update_RxRy(df_update,pose,GSA_angle_WCS_deg)


### 1.4.3 Translate back to the 5DOF frame by undoing the translation from Cell 1.4.1, followed by adding the sMATF delta(X,Y,Z) offset from Cell 1.3

In [16]:
df_update[['X','Y','Z']] += T
df_update.loc[df_update.index[:12],['X','Y','Z']] += update_rot_combined.apply(sMATF_delta.astype(float)) #np.dot(update_rot.as_matrix(),delta)
#ax = plot_poses(df_update)
#plot_sMPA(df_update, ax)
print('Updated poses')
df_update

Updated poses


,X,Y,Z,Rx,Ry,color,uvec_X,uvec_Y,uvec_Z
PDI1,-109.051956,-4.858177,22.8168,-0.908734,10.797742,yellow,242.069695,264.156064,1241.453053
PDI2,111.46801,-4.203048,23.118737,-0.851481,-10.791358,yellow,-241.928257,262.920985,1241.742776
PDI3,39.833031,51.640823,12.523975,4.774464,-3.804747,yellow,-85.740850,142.127114,1281.417277
PDI4,-38.772988,51.530539,12.779639,4.760819,3.954553,yellow,89.111498,142.407041,1281.156205
PR1,0.983694,0.91555,1.107281,0.064242,0.028541,crimson,0.643650,247.417673,1268.213676
sMask,1.283016,115.542889,588.66533,NaN,NaN,g,NaN,NaN,NaN
PATB,88.638936,108.743497,1.454799,0.127647,0.082307,darkblue,1.856176,246.013860,1268.485547
sMPA,1.628426,248.323987,1269.300516,NaN,NaN,purple,0.000332,0.410862,0.911697
sMATF,-0.554449,107.68538,573.74408,0.127647,0.082307,orange,0.820965,108.808982,561.035956
PATB_update,88.738599,107.828385,1.454875,0.154289,0.077648,purple,1.751105,245.424025,1268.599948


# 1.4 >>>>Results<<<<

#### 1.4.1 Converting poses from real space to encoder space

In [17]:
df_update_encoder_space = convert_df_to_encoder_space(df_update)
print('Updated Poses (in encoder space): \n',df_update_encoder_space,'\n\n')

print('Baseline Poses (in encoder space), just for easy reference: \n',pose_encoders)

Updated Poses (in encoder space): 
                X          Y          Z        Rx         Ry
PDI1 -109.024005  -4.813477  22.886407 -0.899929  10.799273
PDI2  111.345782  -4.114898  23.270741 -0.855640 -10.788907
PDI3   39.766761  51.669168  12.654315  4.774411  -3.808898
PDI4  -38.786030  51.543759  12.881630  4.765206   3.951123
PR1     0.954711   0.952518   1.163928  0.066401   0.029266 


Baseline Poses (in encoder space), just for easy reference: 
               X       Y       Z      Rx       Ry
Name.1                                          
PDI1   -109.126  -4.198  21.349 -0.9269  10.8034
PDI2    111.246  -3.505  21.717 -0.8830 -10.7845
PDI3     39.667  52.276  11.079  4.7475  -3.8045
PDI4    -38.887  52.153  11.312  4.7387   3.9557
PR1       0.854   1.555  -0.386  0.0395   0.0340


#### (OPTIONAL) 1.4.2 Display some other tables that may be helpful for diagnostics

In [18]:
print_extra_diagnostics = False
poses_to_display = [val for val in df.index if 'PDI' in val or 'PR' in val]+['sMask', 'sMPA']

if print_extra_diagnostics == True:
    print('Baseline Poses (in real space): \n',df.loc[poses_to_display,['X','Y','Z','Rx','Ry']],'\n\n')
    print('Updated Poses (in real space): \n',df_update.loc[poses_to_display,['X','Y','Z','Rx','Ry']], '\n\n')
    print('Deltas between Baseline/Updated poses (in real space): \n',-df.loc[poses_to_display,['X','Y','Z','Rx','Ry']] + df_update.loc[poses_to_display,['X','Y','Z','Rx','Ry']], '\n\n')
    print('Change in calculated endpoint positions (5DOF frame): \n',compute_endpoint_errors(df,df_update),'\n\n')
    print('Change in calculated endpoint positions (sMPA frame): \n',compute_endpoint_errors_sMPA_frame(df,df_update),'\n\n')
    print('Verify that all poses intersect at the pupil in the same way before and after the update:')
    print(check_pupil_crossing([df,df_update]),'\n\n')
    print('Distance from pose focus to sMPA: \n\n',compute_distance_to_sMPA(df),'\n\n')  
    

### 1.4.3 Write the results to an Excel file in the current directory

In [20]:
write_to_excel = True

if write_to_excel == True:

    charstr='ABCDEFGHIJKLMNOPQRSTUVWXYZ'
    chars=list(charstr)
    nums=[str(i) for i in range(1,26)]

    startrow = 16
    startcol = 1
    df_encoder_space = pd.DataFrame(columns=columns)
    for pose in [val for val in df.index if 'PDI' in val or 'PR' in val]+['PATB']:
        df_encoder_space = pd.concat([df_encoder_space,calculate_encoders_from_5DOF(pd.DataFrame(df.loc[pose]).T)])

    index_names = [val for val in df.index if 'PDI' in val or 'PR' in val]

    gsa_rot = R.from_euler('X',[GSA_angle_WCS_deg], degrees=True)
    # uvec = np.array([0.,0.,dz_update]+p_null_offset)
    rot_uvec = gsa_rot.apply(uvec) #np.dot(gsa_rot.as_matrix(),uvec)

    filename = 'Test output.xlsx'
    sheet1_name = 'baseline'
    sheet2_name = 'update'
    with pd.ExcelWriter(filename) as writer:  
        df.loc[index_names].to_excel(writer, sheet_name=sheet1_name,startrow=startrow,startcol=startcol)
        df_encoder_space.to_excel(writer, sheet_name=sheet1_name,startrow=startrow,startcol=startcol+12)   
        sheet = writer.sheets[sheet1_name]
        sheet[f'{chars[startcol+12]}{startrow}']='Encoders'
        sheet[f'{chars[startcol]}{startrow}']='True positions'
        sheet['B1'] = 'WP2'
        sheet['B2'] = 'Test'
        sheet['A27'] = 'Corresponding field points in GSA'
        sheet['A3'] = 'Track length'
        sheet['B3'] = focal_length+110
    #     sheet['A6'] = 'sMPA offset to WCS'
        newdf = df[df.index=='sMPA'][['X','Y','Z','uvec_X','uvec_Y','uvec_Z']]
        newdf.to_excel(writer,sheet_name=sheet1_name,startrow=4,startcol=0)
        sheet['A10'] = 'GSA angle to WCS'
        sheet['B10'] = GSA_angle_WCS_deg
        newdf = df[df.index=='sMask'][['X','Y','Z']]
        newdf.loc['sMask',['X','Y','Z']] = gsa_rot.inv().apply(newdf.loc['sMask',['X','Y','Z']].values.astype(float))
        newdf.to_excel(writer,sheet_name=sheet1_name,startrow=11,startcol=1)
        endpoints = compute_endpoints(df.loc[index_names],pose_select='P')
        gsa_coordinates = pd.DataFrame(gsa_rot.inv().apply(endpoints),columns=['X','Y','Z'])
        gsa_coordinates.index = index_names
        gsa_coordinates.to_excel(writer,sheet_name=sheet1_name,startrow=27,startcol=1)

        sheet['B41'] = 'ADM measurements'
        p_null_PATB_baseline_encoder.to_excel(writer, sheet_name=sheet1_name,startrow=43,startcol=1)

        df_encoder_space = pd.DataFrame(columns=columns)
        for pose in [val for val in df.index if 'PDI' in val or 'PR' in val]:
            df_encoder_space = pd.concat([df_encoder_space,calculate_encoders_from_5DOF(pd.DataFrame(df_update.loc[pose]).T)])
        df_update.loc[index_names].to_excel(writer, sheet_name=sheet2_name,startrow=startrow,startcol=startcol)
        df_encoder_space.to_excel(writer, sheet_name=sheet2_name,startrow=startrow,startcol=startcol+12)   
        sheet = writer.sheets[sheet2_name]
        sheet[f'{chars[startcol+12]}{startrow}']='Encoders'
        sheet[f'{chars[startcol]}{startrow}']='True positions'
        sheet['B1'] = 'WP2'
        sheet['B2'] = 'Test'
        sheet['A27'] = 'Corresponding field points in GSA'
        sheet['A3'] = 'Track length'
        sheet['B3'] = focal_length+110
    #     sheet['A6'] = 'sMPA offset to WCS'
        newdf = df_update[df_update.index=='sMPA'][['X','Y','Z','uvec_X','uvec_Y','uvec_Z']]
        newdf.to_excel(writer,sheet_name=sheet2_name,startrow=4,startcol=0)
        sheet['A10'] = 'GSA angle to WCS'
        sheet['B10'] = GSA_angle_WCS_deg
        newdf = df_update[df_update.index=='sMask'][['X','Y','Z']]
        newdf.loc['sMask',['X','Y','Z']] = gsa_rot.inv().apply(newdf.loc['sMask',['X','Y','Z']].values.astype(float))
        newdf.to_excel(writer,sheet_name=sheet2_name,startrow=11,startcol=1)
        endpoints = compute_endpoints(df_update.loc[index_names],pose_select='P')
        gsa_coordinates = pd.DataFrame(gsa_rot.inv().apply(endpoints),columns=['X','Y','Z'])
        gsa_coordinates.index = index_names
        gsa_coordinates.to_excel(writer,sheet_name=sheet2_name,startrow=27,startcol=1)

        sheet['B41'] = 'ADM measurements'
        p_null_PATB_update_encoder.to_excel(writer, sheet_name=sheet2_name,startrow=43,startcol=1)
        
        print('**Writing to Excel complete**')


**Writing to Excel complete**


# 5 Updating poses based on results from FDPR

### 5.1 Reading in offset data from the FDPR team's spreadsheet.

In [19]:
#These data will be used to update the position of the calculated endpoints in the sMPA frame
shift_data = pd.read_excel('3rd plateau shifts - circuit 2.xlsx',sheet_name='PSF Shifts',skiprows=0,usecols='C:K')
shift_data = shift_data.iloc[0:6]
shift_data.set_index('camera',inplace=True)
print(shift_data)

        det normal dz (um)  chief ray dz (um)    row  ...  dy (px)  dx (um)  dy (um)
camera                                                ...                           
PR1                     25                 27  244.0  ...     12.0     -117      120
PR3                     -1                  0  238.9  ...     17.1     -130      171
PR4                     37                 45  234.1  ...     21.9      -99      219
PR5                     -4                 -3  240.9  ...     15.1     -161      151
PR2                     62                 64  229.1  ...     26.9     -181      269

[5 rows x 8 columns]


#### 5.1.1 Calculate some transformation variables necessary for converting between the 5DOF and sMPA frame

In [20]:
translation_to_sMPA = df.loc['sMPA',['X','Y','Z']]
rotation_from_sMPA_to_5DOF = R.from_matrix(rotmat_from_2vec(np.array([0,0,1]),df.loc['sMPA',['uvec_X','uvec_Y','uvec_Z']].values.astype(float)))

### 5.2 Calculate the new best-fit poses that minimize the deltas observed by the FDPR team

In [21]:
df_after_fitting_FDPR_shifts,endpoints_residuals_sMPA,best_fit_deltas = pose_update_with_FDPR_results(df,shift_data,focal_length,GSA_angle_WCS_deg,translation_to_sMPA,rotation_from_sMPA_to_5DOF)

Best-fit (X,Y,Z,Rx,Ry) deltas to apply to real-space poses: 
 [ 0.28017783 -0.26922423  0.05370615 -0.00671926 -0.00723571] 

Residuals (in sMPA frame) after incorporating the best-fit (x,y,z,Rx,Ry) shift to all poses: 
           endpt_X       endpt_Y       endpt_Z
PR1  9.067003e-10 -9.290258e-10 -2.169173e-10


C:\Users\ebray\Miniconda3\envs\spyder-env\Lib\site-packages\scipy\optimize\_minpack_py.py:906: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',


### 5.3 Convert new best-fit poses to encoder space

In [22]:
df_after_fitting_FDPR_shifts_in_encoder_space = pd.DataFrame(columns=['X','Y','Z','Rx','Ry'],index=df_after_fitting_FDPR_shifts.index)
for pose in [val for val in pose_encoders.index if ('PR' in val) or ('PD' in val)]:
    df_after_fitting_FDPR_shifts_in_encoder_space.loc[pose] = calculate_encoders_from_5DOF(pd.DataFrame(df_after_fitting_FDPR_shifts.loc[pose]).T).squeeze()

print('New pose encoders, after incorporating the best-fit (x,y,z,Rx,Ry) shift to all poses: \n',df_after_fitting_FDPR_shifts_in_encoder_space)

New pose encoders, after incorporating the best-fit (x,y,z,Rx,Ry) shift to all poses: 
                X          Y          Z        Rx         Ry
PDI1 -108.844772   -4.46747  21.401703 -0.933335  10.796728
PDI2  111.525063   -3.77204  21.768052 -0.888998 -10.791452
PDI3   39.946039  52.008135  11.131332  4.741053  -3.811468
PDI4  -38.606739  51.884016  11.365001  4.731893   3.948553
PR1     1.132305   1.286791  -0.332294  0.033022   0.026725
